<a href="https://colab.research.google.com/github/SerenaYKim/SurveyWeights/blob/master/Opioid_Survey2020_Demographics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from random import sample
from itertools import chain 
import csv
import glob
import os
from functools import reduce

### Load Dataset

In [4]:
urbanrural = pd.read_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/urban_rural_cdc.csv")
race = pd.read_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/race.csv")
pop = pd.read_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/medicaid.csv")
agegender = pd.read_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/age_gender.csv")
surveycrosswalk = pd.read_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/survey_crosswalk.csv")

### Merge and group by "external data reference"

In [5]:
frame = [urbanrural, race, pop, agegender, surveycrosswalk]
merged = reduce(lambda left,right: pd.merge(left, right, on=['fips'],
                                            how='right'), frame)

In [6]:
df = merged

collist = df.columns.values.tolist()
print(collist)

['fips', 'cdc_urban_rural', 'population', 'white', 'black', 'indian', 'asian', 'hawaiian', 'other', 'twoormore', 'threeormore', 'hiswhite', 'hisblack', 'hisindian', 'hisasian', 'hishawaiian', 'hisother', 'histwoormore', 'histhreeormore', 'MedicareCoverageUnder19', 'MedicareCoverageUnder65over', 'medianage', 'SexRatioMalesPer100Females', 'firstname', 'primaryemail', 'externaldatareference', 'unit_id', 'state_name', 'county_name', 'state_fips', 'cnty_fips', 'shared', 'district', 'responded']


In [7]:
#df = df.rename({'pop_x': 'population'}, axis=1, inplace=True)
#df = df.drop(columns=['Population', 'pop'])

In [8]:
df['hispanic'] = df["hiswhite"]+ df["hisblack"] + df["hisindian"] + df["hisasian"] + df["hishawaiian"] + df["hisother"] + df["histwoormore"]  + df["histhreeormore"]
df['location'] = df["county_name"]+", "+ df["state_name"]

In [9]:
df['externaldatareference_st'] = df['externaldatareference'].astype(str)
df['fips_st'] = df['fips'].astype(str)

In [10]:
# population 

series_population = df.groupby("externaldatareference").apply(lambda x: np.sum(x['population']))
df_population = pd.DataFrame(series_population)
df_population = df_population.rename(columns = {0:'population'})

In [11]:
# age 

series_age = df.groupby("externaldatareference").apply(lambda x: np.average(x['medianage'], weights=x['population']))
df_age = pd.DataFrame(series_age)
df_age = df_age.rename(columns={0:"medianage"})

In [12]:
# gender 

series_maletofemale = df.groupby("externaldatareference").apply(lambda x: np.average(x['SexRatioMalesPer100Females'], weights=x['population']))
df_maletofemale = pd.DataFrame(series_maletofemale)
df_maletofemale = df_maletofemale.rename(columns = {0:'SexRatioMalesPer100Females'})

In [13]:
# race

series_white = df.groupby("externaldatareference").apply(lambda x: np.average(x['white'], weights=x['population']))
df_white = pd.DataFrame(series_white)
df_white = df_white.rename(columns = {0:'white'})

series_black = df.groupby("externaldatareference").apply(lambda x: np.average(x['black'], weights=x['population']))
df_black = pd.DataFrame(series_black)
df_black = df_black.rename(columns = {0:'black'})

series_indian = df.groupby("externaldatareference").apply(lambda x: np.average(x['indian'], weights=x['population']))
df_indian = pd.DataFrame(series_indian)
df_indian = df_indian.rename(columns = {0:'indian'})

series_asian = df.groupby("externaldatareference").apply(lambda x: np.average(x['asian'], weights=x['population']))
df_asian = pd.DataFrame(series_asian)
df_asian = df_asian.rename(columns = {0:'asian'})

series_other = df.groupby("externaldatareference").apply(lambda x: np.average(x['other'], weights=x['population']))
df_other = pd.DataFrame(series_other)
df_other = df_other.rename(columns = {0:'other'})

series_twoormore = df.groupby("externaldatareference").apply(lambda x: np.average(x['twoormore'], weights=x['population']))
df_twoormore = pd.DataFrame(series_twoormore)
df_twoormore = df_twoormore.rename(columns = {0:'twoormore'})

series_threeormore = df.groupby("externaldatareference").apply(lambda x: np.average(x['threeormore'], weights=x['population']))
df_threeormore = pd.DataFrame(series_threeormore)
df_threeormore = df_threeormore.rename(columns = {0:'threeormore'})

series_hispanic = df.groupby("externaldatareference").apply(lambda x: np.average(x['hispanic'], weights=x['population']))
df_hispanic = pd.DataFrame(series_hispanic)
df_hispanic = df_hispanic.rename(columns = {0:'hispanic'})

### Concatenate strings 

In [15]:
df_copy_loc = df
df_copy_fip = df

In [16]:
#df.location = np.where(df.location.isnull(),"error",df.location)

df_copy_loc['location'] = df_copy_loc[['externaldatareference', 'location']].groupby(['externaldatareference'])['location'].transform(lambda x : '; '.join(x)) 
df_copy_loc = df_copy_loc[['externaldatareference', 'location']].drop_duplicates()

df_copy_fip['fips_st'] = df_copy_fip[['externaldatareference', 'fips_st']].groupby(['externaldatareference'])['fips_st'].transform(lambda x : '; '.join(x)) 
df_copy_fip = df_copy_fip[['externaldatareference', 'fips_st']].drop_duplicates()


### Merge and export

In [17]:
frame_loc = [df_copy_loc, df_copy_fip]
merged_loc = reduce(lambda left,right: pd.merge(left, right, on=['externaldatareference'],
                                            how='inner'),frame_loc)

In [18]:
df_demographics = pd.concat([df_population, df_age, df_maletofemale, df_white, df_black, df_indian, df_asian, df_other, df_twoormore, df_threeormore, df_hispanic], axis=1)
df_demographics = df_demographics.reset_index()

In [19]:
frames = [df_demographics, merged_loc]
demographics_master = reduce(lambda left,right: pd.merge(left, right, on=['externaldatareference'],
                                            how='inner'),frames)

In [20]:
demographics_master.to_csv("/content/drive/MyDrive/G10-Opioid/dataDemographics/demographics_master.csv")

### References 

* https://datatofish.com/convert-pandas-series-to-dataframe/ 

* https://www.geeksforgeeks.org/concatenate-strings-from-several-rows-using-pandas-groupby/ 

* https://stackoverflow.com/questions/55998672/how-to-fix-this-typeerror-sequence-item-0-expected-str-instance-float-found